In [ ]:
!apt update
!apt install chromium-chromedriver
!pip install selenium
!pip install bs4

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

In [3]:
import json
import numpy as np
import pandas as pd         
import ctypes

In [4]:
hospitals = pd.read_csv('https://raw.githubusercontent.com/bhavya-kashyap/web-scrapping/main/GA_Hospitals.csv')
baseurl = 'https://freida.ama-assn.org/institution/'
URLS = []
for index, i in hospitals.iterrows():
  URLS.append(str(baseurl + str(i['H.No.'])))
hospitals['URLS'] = URLS

hospitals = hospitals.fillna('')
hospitals['Med_center'] = hospitals['Med_center'] + " " + hospitals['Unnamed: 2']
hospitals = hospitals.drop('Unnamed: 2', axis=1)


hospitals.head()

,H.No.,Med_center,URLS
0,120127,Abraham Infectious Disease Associates,https://freida.ama-assn.org/institution/120127
1,128070,AG Rhodes Homes,https://freida.ama-assn.org/institution/128070
2,129528,Air Methods Air Life Georgia,https://freida.ama-assn.org/institution/129528
3,129505,Albany Arthritis and Orthopedic Center,https://freida.ama-assn.org/institution/129505
4,120063,Alfred E Beach High School (APEX Program by G...,https://freida.ama-assn.org/institution/120063


In [7]:
def driversetup():
    options = webdriver.ChromeOptions()
    #run Selenium in headless mode
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    #overcome limited resource problems
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("lang=en")
    #open Browser in maximized mode
    options.add_argument("start-maximized")
    #disable infobars
    options.add_argument("disable-infobars")
    #disable extension
    options.add_argument("--disable-extensions")
    options.add_argument("--incognito")
    options.add_argument("--disable-blink-features=AutomationControlled")
    
    driver = webdriver.Chrome(options=options)

    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")

    return driver

def pagesource(url, driver):
    driver = driver
    driver.get(url)
    time.sleep(10)
    soup = BeautifulSoup(driver.page_source)
    #driver.close()
    return soup

d = driversetup()
s = pagesource('https://freida.ama-assn.org/institution/120139', d)
s

<html lang="en"><head>
<meta charset="utf-8"/>
<title>Browse Emory University Hospital's ACGME Programs | 120139 | FREIDA™</title>
<base href="/"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="Find affiliated programs, clinical environment details, and additional info about Emory University Hospital in Atlanta, GA." name="description"/>
<!-- Main Link Tags  -->
<link href="../assets/icons/favicon-16.png" rel="icon" sizes="16x16" type="image/png"/>
<link href="../assets/icons/favicon-32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="../assets/icons/favicon-48.png" rel="icon" sizes="48x48" type="image/png"/>
<!-- iOS  -->
<link href=" " rel="apple-touch-icon"/>
<link href="../assets/icons/touch-icon-ipad.png" rel="apple-touch-icon" sizes="76x76"/>
<link href="../assets/icons/touch-icon-iphone-retina.png" rel="apple-touch-icon" sizes="120x120"/>
<link href="../assets/icons/touch-icon-ipad-retina.png" rel="apple-touch-icon" sizes="152x1

In [27]:
a = list(s.findAll('div', class_="medical-schools__school ng-star-inserted"))
for i in range(len(a)):
  name = a[i].findAll('small', class_='bold')[0].text
  relationship = a[i].findAll('div', class_='')[0].text
  if 'Major' in relationship:
    print(name, ': Major')
  if 'Limited' in relationship:
    print(name, ': Limited')
#mcl.append(name)

Emory Univ Sch of Med : Major
Morehouse Sch of Med : Limited
West Virginia Sch of Osteopathic Med : Limited


In [29]:
mc = []
mcl = []
add = []
d = driversetup()
for index, i in hospitals.iterrows():
  print(index)
  url = i['URLS']
  s = pagesource(url, d)
  med_school = list(s.findAll('small', class_="bold"))
  address = list(s.findAll('div', class_="details__address ng-star-inserted"))
  address[0].br.string = ', '
  if med_school == []:
    mc.append("")
    mcl.append("")
  elif len(med_school) == 1:
    a = list(s.findAll('div', class_="medical-schools__school ng-star-inserted"))
    for i in range(len(a)):
      name = a[i].findAll('small', class_='bold')[0].text
      relationship = a[i].findAll('div', class_='')[0].text
      if 'Major' in relationship:
        mc.append(name)
        mcl.append("")
      elif 'Limited' in relationship:
        mcl.append(name)
        mc.append("")
  elif len(med_school) > 1:
    temp_major = []
    temp_limited = []
    a = list(s.findAll('div', class_="medical-schools__school ng-star-inserted"))
    for i in range(len(a)):
      name = a[i].findAll('small', class_='bold')[0].text
      relationship = a[i].findAll('div', class_='')[0].text
      if 'Major' in relationship:
        temp_major.append(name)
      if 'Limited' in relationship:
        temp_limited.append(name)
      mc.append(temp_major)
      mcl.append(temp_limited

  if address != []:
    add.append(address[0].text)
  else:
    add.append("")
  print(mc[index], mcl[index], add[index])


hospitals['Med_School_Major'] = mc
hospitals['Med_School_Limited'] = mcl
hospitals['Address'] = add
hospitals.head(10)

0
  2024 E Pinetree Blvd, Thomasville, GA 31792
1
  3715 Northside Pkwy, Atlanta, GA 30327
2
  635 Regional Airport Rd, Carrollton, GA 30117
3
  2100 Palmyra Rd, Albany, GA 31701
4
  3001 Hopkins St, Savannah, GA 31405
5
  1120 15th St, Augusta, GA 30912
6
 Morehouse Sch of Med 250 Williams St NW, Atlanta, GA 30303
7
  112 Mimosa Dr, Thomasville, GA 31792
8
  915 Gordon Ave, Thomasville, GA 31799
9
  2705 E Pinetree Blvd, Thomasville, GA 31792
10
  961 Smoky Mountain Springs, Gainesville, GA 30501
11
  895 Canton Rd, Marietta, GA 30060
12
  3193 Howell Mill Rd, Atlanta, GA 30327
13
  232 19th St NW, Atlanta, GA 30363
14
  1955 Textile Way, Gainesville, GA 30501
15
  975 Johnson Ferry Rd, Atlanta, GA 30342
16
  3200 Downwood Circle NW, Atlanta, GA 30327
17
  3200 Downwood Circle NW, Atlanta, GA 30327
18
  3200 Downwood Circle NW, Atlanta, GA 30327
19
  235 W Wieuca Rd NE, Atlanta, GA 30342
20
Mercer Univ Sch of Med  777 Hemlock St, Macon, GA 31208-6000
21
  3421 Mile Padgett Hwy, August

ValueError: ignored

In [30]:
#print(len(mc),len(mcl), len(add))
#hospitals.head(5)

327 327 290


In [17]:
#hospitals.to_csv('Hospitals.csv')
#hospitals = pd.read_csv("Hospitals.csv")
#hospitals.head(10)

,Unnamed: 0,H.No.,Med_center,URLS,Med_School,Address
0,0,120127,Abraham Infectious Disease Associates,https://freida.ama-assn.org/institution/120127,NaN,"2024 E Pinetree Blvd, Thomasville, GA 31792"
1,1,128070,AG Rhodes Homes,https://freida.ama-assn.org/institution/128070,NaN,"3715 Northside Pkwy, Atlanta, GA 30327"
2,2,129528,Air Methods Air Life Georgia,https://freida.ama-assn.org/institution/129528,NaN,"635 Regional Airport Rd, Carrollton, GA 30117"
3,3,129505,Albany Arthritis and Orthopedic Center,https://freida.ama-assn.org/institution/129505,NaN,"2100 Palmyra Rd, Albany, GA 31701"
4,4,120063,Alfred E Beach High School (APEX Program by G...,https://freida.ama-assn.org/institution/120063,NaN,"3001 Hopkins St, Savannah, GA 31405"
5,5,128064,Ambulatory Care Center Medical College of Geo...,https://freida.ama-assn.org/institution/128064,NaN,"1120 15th St, Augusta, GA 30912"
6,6,128103,American Cancer Society Georgia,https://freida.ama-assn.org/institution/128103,Morehouse Sch of Med,"250 Williams St NW, Atlanta, GA 30303"
7,7,120130,Archbold Gastroenterology Group,https://freida.ama-assn.org/institution/120130,NaN,"112 Mimosa Dr, Thomasville, GA 31792"
8,8,128069,Archbold Medical Center,https://freida.ama-assn.org/institution/128069,NaN,"915 Gordon Ave, Thomasville, GA 31799"
9,9,120128,Archbold Pulmonary Medicine and Critical Care...,https://freida.ama-assn.org/institution/120128,NaN,"2705 E Pinetree Blvd, Thomasville, GA 31792"
